In [3]:
import os
import numpy as np
import pandas as pd
from pycocotools.coco import COCO
from PIL import Image
from albumentations.pytorch import ToTensorV2
import albumentations as al
import cv2

In [4]:
# Data augmentation method

transform = al.Compose([
    al.Flip(p=0.1),  # flip (horizontal or vertical)
    al.ShiftScaleRotate(p=0.2),  # shift, scale, and rotate
    al.RandomBrightnessContrast(p=0.2),  # brightness and contrast
    ToTensorV2()
], is_check_shapes=False)

In [5]:
# Extract and augment training data

json_file = 'dataset/train/_annotations.coco.json' 
images_dir = 'dataset/train'

# Initialize COCO object
coco = COCO(json_file)
image_ids = coco.getImgIds()

data = []

# Collect features for clustering
features = []

for annotation in coco.anns.values():

    img_info = coco.loadImgs(annotation['image_id'])[0]
    img_path = os.path.join(images_dir, img_info['file_name'])

    # Load image and segmentation mask
    image = np.array(Image.open(img_path).convert('RGB'))
    segmentation = np.array(annotation['segmentation'])

    # Apply data augmentation
    augmented = transform(image=image, mask=segmentation)
    image = augmented['image']
    segmentation = augmented['mask']

    # Extract and store image and segmentation features
    combined_features = np.concatenate([image.flatten(), segmentation.flatten()])
    features.append(combined_features)

    # Append data as dictionary
    data.append({
        'image': image,
        'segmentation': segmentation
    })

df = pd.DataFrame(data)

loading annotations into memory...
Done (t=0.32s)
creating index...
index created!


In [6]:
print(df.head())

                                               image  \
0  [[[tensor(135, dtype=torch.uint8), tensor(133,...   
1  [[[tensor(53, dtype=torch.uint8), tensor(55, d...   
2  [[[tensor(128, dtype=torch.uint8), tensor(127,...   
3  [[[tensor(180, dtype=torch.uint8), tensor(178,...   
4  [[[tensor(133, dtype=torch.uint8), tensor(133,...   

                                        segmentation  
0  [[tensor(205.5000, dtype=torch.float64), tenso...  
1  [[tensor(266.3330, dtype=torch.float64), tenso...  
2  [[tensor(535.7040, dtype=torch.float64), tenso...  
3  [[tensor(253., dtype=torch.float64), tensor(25...  
4  [[tensor(130., dtype=torch.float64), tensor(19...  


In [7]:
# Save to file
df.to_csv("TRAIN_SET.csv")